In [2]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [21]:
df = pd.read_csv('seatsurveymodel.csv')

In [22]:
df.columns

Index(['participant.id_in_session', 'participant.code', 'participant.label',
       'participant._is_bot', 'participant._index_in_pages',
       'participant._max_page_index', 'participant._current_app_name',
       'participant._current_page_name', 'participant.time_started',
       'participant.visited',
       ...
       'player.legroom_21b', 'player.choice_21', 'group.id_in_subsession',
       'subsession.round_number', 'session.code', 'session.label',
       'session.mturk_HITId', 'session.mturk_HITGroupId', 'session.comment',
       'session.is_demo'],
      dtype='object', length=277)

In [28]:
df.drop(['participant.code','participant.label','participant._is_bot',
        'participant._current_app_name','participant._current_page_name','participant.visited',
        'participant.mturk_worker_id','participant.mturk_assignment_id','participant.payoff',
        'player.id_in_group','player.role','player.payoff','group.id_in_subsession','subsession.round_number',
        'session.code','session.label','session.mturk_HITId','session.mturk_HITGroupId','session.comment',
        'session.is_demo'], inplace = True, axis = 1
       )

In [29]:
df.columns

Index(['participant.id_in_session', 'participant._index_in_pages',
       'participant._max_page_index', 'participant.time_started',
       'player.page_sequence', 'player.choice1', 'player.choice2',
       'player.distance_1', 'player.time_1', 'player.price_1',
       ...
       'player.price_21', 'player.row_21a', 'player.row_21b', 'player.col_21a',
       'player.col_21b', 'player.price_21a', 'player.price_21b',
       'player.legroom_21a', 'player.legroom_21b', 'player.choice_21'],
      dtype='object', length=257)

In [30]:
newdf = pd.DataFrame(np.repeat(df.values,3,axis=0))
newdf.columns = df.columns
print(newdf)

    participant.id_in_session participant._index_in_pages  \
0                           2                          23   
1                           2                          23   
2                           2                          23   
3                           1                          23   
4                           1                          23   
..                        ...                         ...   
256                       183                          23   
257                       183                          23   
258                       186                          23   
259                       186                          23   
260                       186                          23   

    participant._max_page_index          participant.time_started  \
0                            22  2021-04-15 09:30:39.667824+00:00   
1                            22  2021-04-15 09:30:39.667824+00:00   
2                            22  2021-04-15 09:30:39.667824+